# Training an activity classifier

This notebook was used to train the activity classification model **GestureClassifier.mlmodel** supplied with the book Machine Learning by Tutorials.

In [1]:
'''
/// Copyright (c) 2018 Razeware LLC
/// 
/// Permission is hereby granted, free of charge, to any person obtaining a copy
/// of this software and associated documentation files (the "Software"), to deal
/// in the Software without restriction, including without limitation the rights
/// to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
/// copies of the Software, and to permit persons to whom the Software is
/// furnished to do so, subject to the following conditions:
/// 
/// The above copyright notice and this permission notice shall be included in
/// all copies or substantial portions of the Software.
/// 
/// Notwithstanding the foregoing, you may not use, copy, modify, merge, publish,
/// distribute, sublicense, create a derivative work, and/or sell copies of the
/// Software in any work that is designed, intended, or marketed for pedagogical or
/// instructional purposes related to programming, coding, application development,
/// or information technology.  Permission for such use, copying, modification,
/// merger, publication, distribution, sublicensing, creation of derivative works,
/// or sale is expressly withheld.
/// 
/// THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
/// IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
/// FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
/// AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
/// LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
/// OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
/// THE SOFTWARE.
'''

'\n/// Copyright (c) 2018 Razeware LLC\n/// \n/// Permission is hereby granted, free of charge, to any person obtaining a copy\n/// of this software and associated documentation files (the "Software"), to deal\n/// in the Software without restriction, including without limitation the rights\n/// to use, copy, modify, merge, publish, distribute, sublicense, and/or sell\n/// copies of the Software, and to permit persons to whom the Software is\n/// furnished to do so, subject to the following conditions:\n/// \n/// The above copyright notice and this permission notice shall be included in\n/// all copies or substantial portions of the Software.\n/// \n/// Notwithstanding the foregoing, you may not use, copy, modify, merge, publish,\n/// distribute, sublicense, create a derivative work, and/or sell copies of the\n/// Software in any work that is designed, intended, or marketed for pedagogical or\n/// instructional purposes related to programming, coding, application development,\n/// or i

In [2]:
import turicreate as tc
import activity_detector_utils as utils

# Load clean datasets

In [4]:
train_sf = tc.SFrame("data/cleaned_train_sframe")
valid_sf = tc.SFrame("data/cleaned_valid_sframe")
test_sf = tc.SFrame("data/cleaned_test_sframe")

### Optional: Split a validation set from your training set

This step is **not** necessary because we already have a separate validation set. This project does not use the results of the following cell, but it's here as an example to show you how to do it *if* you ever want to in your own projects.

In [5]:
train, valid = tc.activity_classifier.util.random_split_by_session(train_sf, session_id='sessionId', fraction=0.9)

In [6]:
utils.count_activities(train)
utils.count_activities(valid)

+----------+--------+-------+
| activity | userId | Count |
+----------+--------+-------+
| chop_it  |  u_01  |   29  |
| chop_it  |  u_02  |   33  |
| drive_it |  u_01  |   34  |
| drive_it |  u_02  |   33  |
| rest_it  |  u_01  |   95  |
| rest_it  |  u_02  |  100  |
| shake_it |  u_01  |   32  |
| shake_it |  u_02  |   34  |
+----------+--------+-------+
[8 rows x 3 columns]

+----------+--------+-------+
| activity | userId | Count |
+----------+--------+-------+
| chop_it  |  u_01  |   7   |
| chop_it  |  u_02  |   3   |
| drive_it |  u_01  |   2   |
| drive_it |  u_02  |   3   |
| rest_it  |  u_01  |   13  |
| rest_it  |  u_02  |   8   |
| shake_it |  u_01  |   4   |
| shake_it |  u_02  |   2   |
+----------+--------+-------+
[8 rows x 3 columns]



# Verify your model correctness by overfitting

This step is to ensure the dataset and model are applicable to the problem, and the model is implemented properly. The following cell grabs a random 5% of the training data and trains an activity classifier model with it to ensure it learns successfully. You can see the training accuracy is quite high – at times 100% – which is a good indication that the model should be able to learn from your dataset.

In [7]:
tiny_train, _ = tc.activity_classifier.util.random_split_by_session(train_sf, session_id='sessionId', fraction=0.05)

tc.activity_classifier.create(
    dataset=tiny_train, session_id='sessionId', target='activity', 
    features=["rotX", "rotY", "rotZ", "accelX", "accelY", "accelZ"],
    prediction_window=20, max_iterations=50)

The dataset has less than the minimum of 100 sessions required for train-validation split. Continuing without validation set


Pre-processing 13114 samples...

Using sequences of size 400 for model creation.

Processed a total of 11 sessions.

Using GPU to create model (AMD Radeon HD - FirePro D700)
+----------------+----------------+----------------+----------------+
| Iteration      | Train Accuracy | Train Loss     | Elapsed Time   |
+----------------+----------------+----------------+----------------+
| 1              | 0.232          | 1.272          | 5.7            |
| 2              | 0.746          | 0.831          | 5.8            |
| 3              | 0.762          | 0.743          | 5.8            |
| 4              | 0.805          | 0.673          | 5.9            |
| 5              | 0.859          | 0.576          | 5.9            |
| 6              | 0.846          | 0.512          | 6.0            |
| 7              | 0.903          | 0.428          | 6.0            |
| 8              | 0.911          | 0.356          | 6.1            |
| 9              | 0.913          | 0.283          | 6.1            |
| 10             | 0.920          | 0.268          | 6.2            |
| 11             | 0.949         

Class                                    : ActivityClassifier

Schema
------
Number of examples                       : 13114
Number of sessions                       : 11
Number of classes                        : 4
Number of feature columns                : 6
Prediction window                        : 20

Training summary
----------------
Log-likelihood                           : 0.0296
Training time (sec)                      : 8.4815

# Train the model

You'll get different results every time you run this training cell, because there's some randomness built into the training process. You may get slightly better or worse results, but they should be pretty close to these.

In [8]:
# Create an activity classifier
model = tc.activity_classifier.create(
    dataset=train_sf, session_id='sessionId', target='activity', 
    features=["rotX", "rotY", "rotZ", "accelX", "accelY", "accelZ"],
    prediction_window=20, validation_set=valid_sf, max_iterations=50)

Pre-processing 235057 samples...

Using sequences of size 400 for model creation.

Processed a total of 216 sessions.

Using GPU to create model (AMD Radeon HD - FirePro D700)
+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+
| Iteration           | Train Accuracy      | Train Loss          | Validation Accuracy | Validation Loss     | Elapsed Time        |
+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+
| 1                   | 0.590               | 0.939               | 0.721               | 0.958               | 0.4                 | 
| 2                   | 0.838               | 0.480               | 0.942               | 0.351               | 0.9                 | 
| 3                   | 0.914               | 0.316               | 0.954               | 0.148               | 1.3                 | 
| 4                   | 0.941               | 0.237               | 0.960               | 0.125               | 1.8                 | 
|

In [9]:
model.summary()

Class                                    : ActivityClassifier

Schema
------
Number of examples                       : 235057
Number of sessions                       : 216
Number of classes                        : 4
Number of feature columns                : 6
Prediction window                        : 20

Training summary
----------------
Log-likelihood                           : 0.0252
Training time (sec)                      : 23.7814



## Evaluate trained model

In [10]:
metrics = model.evaluate(test_sf)

In [11]:
print(f"Accuracy: {metrics['accuracy']}")

Accuracy: 0.9748703362891082


In [12]:
print(metrics)

{'accuracy': 0.9748703362891082, 'auc': 0.9969528472683438, 'precision': 0.9784590236279842, 'recall': 0.9695791541946717, 'f1_score': 0.9737156348967198, 'log_loss': 0.11922604903345482, 'confusion_matrix': Columns:
	target_label	str
	predicted_label	str
	count	int

Rows: 11

Data:
+--------------+-----------------+-------+
| target_label | predicted_label | count |
+--------------+-----------------+-------+
|   shake_it   |     rest_it     |  108  |
|   rest_it    |     drive_it    |   17  |
|   drive_it   |     drive_it    |  5423 |
|   chop_it    |     shake_it    |  156  |
|   rest_it    |     rest_it     | 13185 |
|   chop_it    |     rest_it     |  148  |
|   rest_it    |     chop_it     |   29  |
|   shake_it   |     shake_it    |  5355 |
|   drive_it   |     rest_it     |  204  |
|   chop_it    |     chop_it     |  5171 |
+--------------+-----------------+-------+
[11 rows x 3 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns

In [13]:
metrics['confusion_matrix'].print_rows(num_rows=11)

+--------------+-----------------+-------+
| target_label | predicted_label | count |
+--------------+-----------------+-------+
|   shake_it   |     rest_it     |  108  |
|   rest_it    |     drive_it    |   17  |
|   drive_it   |     drive_it    |  5423 |
|   chop_it    |     shake_it    |  156  |
|   rest_it    |     rest_it     | 13185 |
|   chop_it    |     rest_it     |  148  |
|   rest_it    |     chop_it     |   29  |
|   shake_it   |     shake_it    |  5355 |
|   drive_it   |     rest_it     |  204  |
|   chop_it    |     chop_it     |  5171 |
|   rest_it    |     shake_it    |   89  |
+--------------+-----------------+-------+
[11 rows x 3 columns]



## Save the trained model

Export the model to Core ML and save a copy you can reload here if you want to do anything else with it (e.g. test it on a different dataset)

In [14]:
model.export_coreml("GestureClassifier.mlmodel")
model.save("GestureClassifier")